<h1> Microsoft 365 - GraphQL </h1>

In [ ]:
import re
import pandas as pd
import json
import datetime as dt
import logging

import msal
import requests
from requests.utils import CaseInsensitiveDict

In [ ]:

import os
import hvac

def read_secret(mount_point, path):
    client = hvac.Client(url=os.environ.get("VAULT_ADDR"))
    resp = client.kv.v2.read_secret(mount_point=mount_point, path=f'/{path}')
    secret = resp['data']['data']

    return secret


In [ ]:
secrets = read_secret("azuread", "app_01")

In [ ]:
scopes = [
    "Mail.Read.Shared",
    "Mail.ReadWrite.Shared",
    "Mail.Send.Shared",
]

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

In [ ]:
# token URL
authorityUrl = f"https://login.microsoftonline.com/{secrets['tenant_id']}"
# scope = list("https://graph.microsoft.com/.default")
scope = f"{secrets['application_id']}/.default"
endpoint = "https://graph.microsoft.com/v1.0/users"
tokenUrl = f"https://login.microsoftonline.com/{secrets['tenant_id']}/oauth2/token"
api_version = 'v1.0'

In [ ]:
client  = msal.ConfidentialClientApplication(
    client_id=secrets['application_id'],
    client_credential=secrets['client_secret'],
    authority=authorityUrl,
)

In [ ]:
result = client.acquire_token_silent(scopes=scopes,account=None)

if not result:
    logging.info("No suitable token exists in cache. Let's get a new one from AAD.")
    result = client.acquire_token_for_client(
        scopes=scope,
    )

access_token = result['access_token']

In [ ]:
user_mailbox = 'reports@example.co'

In [ ]:
date_received = dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S')

In [ ]:
resource = "messages"

In [ ]:
mail_subject = ""
mail_sender = ""
mail_user = ""

In [ ]:
query = """query

    }"""

In [ ]:
# request content response
request_url = f"https://graph.microsoft.com/v1.0/users/{user_mailbox}/messages"
# request_url = f"https://graph.microsoft.com/v1.0/gmcwilliams@example.co/{resource}"
# https://graph.microsoft.com/v1.0/me/messages

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)
print(c_dict)

In [ ]:
resource = 'reports@example.co'

In [ ]:
query = {}
query['select'] = 'Inbox'
query['top'] = 10

In [ ]:
# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# request content response
request_url = f"https://graph.microsoft.com/v1.0/{resource}?{query}"

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)
print(c_dict)

In [ ]:
df = pd.DataFrame(c_dict['value'])

In [ ]:
df

In [ ]:
df.to_csv('[REDACTED]/.csv')